In [1]:
import torch, ultralytics
import os, glob, yaml, random, shutil
from pathlib import Path
from collections import Counter, defaultdict
from PIL import Image, ImageDraw
from IPython.display import display
from ultralytics import YOLO, RTDETR
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
Data_Folder = "/Users/Admin/Desktop/12/pothole/pothole_only"
Results_Folder = "/Users/Admin/Desktop/12/pothole/results"
yaml_to_use = "/Users/Admin/Desktop/12/pothole/pothole_only/data.yaml"

In [3]:
DATA_ROOT = Path(Data_Folder)
YAML_PATH = Path(yaml_to_use)

In [4]:
# Load data.yaml
with open(YAML_PATH, "r", encoding="utf-8") as f:
    ycfg = yaml.safe_load(f)


In [5]:
import gc
gc.collect()
torch.cuda.empty_cache()
torch.backends.cudnn.benchmark = True

In [6]:
ycfg

{'train': '../train/images',
 'val': '../valid/images',
 'nc': 1,
 'names': ['pothole']}

In [7]:
# count images
print("Train images:", len(list((Path(DATA_ROOT)/"train/images").glob("*.jpg"))))
print("Train labels:", len(list((Path(DATA_ROOT)/"train/labels").glob("*.txt"))))
print("Val images:", len(list((Path(DATA_ROOT)/"valid/images").glob("*.jpg"))))
print("Val labels:", len(list((Path(DATA_ROOT)/"valid/labels").glob("*.txt"))))

Train images: 1581
Train labels: 1581
Val images: 396
Val labels: 396


In [8]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())
print(torch.cuda.device_count())

2.7.1+cu118
11.8
90100
True
1


Train

In [9]:
# Load model

model = YOLO("yolo11n.pt")
model.info()

YOLO11n summary: 181 layers, 2,624,080 parameters, 0 gradients, 6.6 GFLOPs


(181, 2624080, 0, 6.614336)

In [10]:
model.to("cuda")

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

In [11]:
results = model.train(
    data=yaml_to_use,
    epochs=150,           
    imgsz=640,            
    batch=8,            
    lr0=4e-5,             
    lrf=0.002,
    close_mosaic=5,
    freeze=10,           
    cos_lr=True,
    amp=True,
    optimizer="AdamW",
    mosaic=0.3,
    mixup=0.2,
    copy_paste=0.2,
    patience=10,
    project=Results_Folder,
    device=0,
)


New https://pypi.org/project/ultralytics/8.3.221 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.213  Python-3.10.0 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=5, cls=0.5, compile=False, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/Users/Admin/Desktop/12/pothole/pothole_only/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=4e-05, lrf=0.002, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=0.3, multi_scale=False, na

# **Eval**

In [24]:
# RT-DETR evaluation
yolo_path = "/Users/Admin/Desktop/12/pothole/results/train14/weights/best.pt"
yolo_model = YOLO(yolo_path)
yolo_metrics = yolo_model.val(data=YAML_PATH, save=True, project = Results_Folder)

Ultralytics 8.3.213  Python-3.10.0 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 9.57.3 MB/s, size: 164.9 KB)
val: Scanning C:\Users\Admin\Desktop\12\pothole\pothole_only\valid\labels.cache... 395 images, 0 backgrounds, 1 corrupt: 100% ━━━━━━━━━━━━ 396/396 396.6Kit/s 0.0s
val: C:\Users\Admin\Desktop\12\pothole\pothole_only\valid\images\pothole_1870.jpg: ignoring corrupt image/label: invalid image format GIF. Supported formats are:
images: {'pfm', 'tif', 'webp', 'bmp', 'tiff', 'dng', 'mpo', 'heic', 'jpg', 'png', 'jpeg'}
videos: {'gif', 'mp4', 'mpeg', 'mpg', 'avi', 'mov', 'm4v', 'webm', 'asf', 'wmv', 'ts', 'mkv'}
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 25/25 3.1it/s 8.0s<0.1s
                   all        395       1394      0.785       0.69       0.78      0.516
Speed: 0

In [72]:
yolo_metrics.results_dict

{'metrics/precision(B)': 0.9045660031373927,
 'metrics/recall(B)': 0.8895705521472392,
 'metrics/mAP50(B)': 0.9236614504305913,
 'metrics/mAP50-95(B)': 0.6300689099975645,
 'fitness': 0.6300689099975645}

# **Inference**

In [12]:
weights_path = "/Users/Admin/Desktop/12/pothole/results/train14/weights/best.pt"
model = YOLO(weights_path)

In [13]:
sample_data = "/Users/Admin/Desktop/12/samples"

**SAMPLE 1**

In [16]:
# SAMPLE 1 IMAGE
img_path = os.path.join(sample_data, "test11.jpg")

# Inference on image
yolo_result_pred = model.predict(
    source=img_path,
    conf=0.7,                      # confidence threshold
    iou=0.7,
    imgsz=640,
    save=True,                     # save annotated image
    show=True,
    project = "/Users/Admin/Desktop/12/results_2",
     name="inference",
    exist_ok=True
)

saved_path = os.path.join("/Users/Admin/Desktop/12/results_2/inference/test11.jpg")
img = Image.open(saved_path)
plt.imshow(img)
plt.axis("off")
plt.show()


image 1/1 c:\Users\Admin\Desktop\12\samples\test11.jpg: 416x640 1 pothole, 671.8ms
Speed: 1.3ms preprocess, 671.8ms inference, 2.4ms postprocess per image at shape (1, 3, 416, 640)
Results saved to C:\Users\Admin\Desktop\12\results_2\inference


<Figure size 640x480 with 1 Axes>

In [20]:
# SAMPLE 2 IMAGE
img_path = os.path.join(sample_data, "test12.jpg")

# Inference on image
yolo_result_pred = model.predict(
    source=img_path,
    conf=0.7,                      # confidence threshold
    iou=0.7,
    imgsz=640,
    save=True,                     # save annotated image
    show=True,
    project = "/Users/Admin/Desktop/12/results_2",
    name="inference",
    exist_ok=True
)

saved_path = os.path.join("/Users/Admin/Desktop/12/results_2/inference/test12.jpg")
img = Image.open(saved_path)
plt.imshow(img)
plt.axis("off")
plt.show()


image 1/1 c:\Users\Admin\Desktop\12\samples\test12.jpg: 448x640 7 potholes, 138.8ms
Speed: 2.1ms preprocess, 138.8ms inference, 9.4ms postprocess per image at shape (1, 3, 448, 640)
Results saved to C:\Users\Admin\Desktop\12\results_2\inference


<Figure size 640x480 with 1 Axes>

In [21]:
# SAMPLE 3 IMAGE
img_path = os.path.join(sample_data, "test13.jpg")

# Inference on image
yolo_result_pred = model.predict(
    source=img_path,
    conf=0.7,                      # confidence threshold
    iou=0.7,
    imgsz=640,
    save=True,                     # save annotated image
    show=True,
    project = "/Users/Admin/Desktop/12/results_2",
    name="inference",
    exist_ok=True
)

saved_path = os.path.join("/Users/Admin/Desktop/12/results_2/inference/test13.jpg")
img = Image.open(saved_path)
plt.imshow(img)
plt.axis("off")
plt.show()


image 1/1 c:\Users\Admin\Desktop\12\samples\test13.jpg: 480x640 1 pothole, 635.5ms
Speed: 1.1ms preprocess, 635.5ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
Results saved to C:\Users\Admin\Desktop\12\results_2\inference


<Figure size 640x480 with 1 Axes>

In [22]:
# SAMPLE 4 IMAGE
img_path = os.path.join(sample_data, "test14.jpg")

# Inference on image
yolo_result_pred = model.predict(
    source=img_path,
    conf=0.7,                      # confidence threshold
    iou=0.7,
    imgsz=640,
    save=True,                     # save annotated image
    show=True,
    project = "/Users/Admin/Desktop/12/results_2",
    name="inference",
    exist_ok=True
)

saved_path = os.path.join("/Users/Admin/Desktop/12/results_2/inference/test14.jpg")
img = Image.open(saved_path)
plt.imshow(img)
plt.axis("off")
plt.show()


image 1/1 c:\Users\Admin\Desktop\12\samples\test14.jpg: 480x640 (no detections), 8.3ms
Speed: 2.0ms preprocess, 8.3ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
Results saved to C:\Users\Admin\Desktop\12\results_2\inference


<Figure size 640x480 with 1 Axes>